In [1]:
#IMport necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#import the smartphone data
df = pd.read_csv('./smartphone_cleaned_v5.csv')
df.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
0,oneplus,OnePlus 11 5G,54999,89.0,True,True,False,snapdragon,8.0,3.2,...,6.70,120,1440 x 3216,3,1.0,android,50.0,16.0,0,NaN
1,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,True,False,False,snapdragon,8.0,2.2,...,6.59,120,1080 x 2412,3,1.0,android,64.0,16.0,1,1024.0
2,samsung,Samsung Galaxy A14 5G,16499,75.0,True,False,False,exynos,8.0,2.4,...,6.60,90,1080 x 2408,3,1.0,android,50.0,13.0,1,1024.0
3,motorola,Motorola Moto G62 5G,14999,81.0,True,False,False,snapdragon,8.0,2.2,...,6.55,120,1080 x 2400,3,1.0,android,50.0,16.0,1,1024.0
4,realme,Realme 10 Pro Plus,24999,82.0,True,False,False,dimensity,8.0,2.6,...,6.70,120,1080 x 2412,3,1.0,android,108.0,16.0,0,NaN


In [3]:
#Pre=Processing
df = df.sort_values(by='model',ascending=True)
df["SN"] = list(range(len(df.index)))
df=df.set_index('SN')

#Remove the NA values   
df =df.ffill(axis=0)
#Remove the errors where no memory slot but the NA filling method has given storage
df['extended_upto'] = np.where(df['extended_memory_available']==0,0,df['extended_upto'])
df.fillna({'fast_charging':22},inplace=True) #ffill didn't work for iphone 11 and iphone 11 pro in the 1st run of code. so it manually for 2 mobiles
df.fillna({'extended_upto':0},inplace=True) #ffill didn't work for iphone 11 and iphone 11 pro in the 1st run of code. so it manually for 2 mobiles
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980 entries, 0 to 979
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   brand_name                 980 non-null    object 
 1   model                      980 non-null    object 
 2   price                      980 non-null    int64  
 3   rating                     980 non-null    float64
 4   has_5g                     980 non-null    bool   
 5   has_nfc                    980 non-null    bool   
 6   has_ir_blaster             980 non-null    bool   
 7   processor_brand            980 non-null    object 
 8   num_cores                  980 non-null    float64
 9   processor_speed            980 non-null    float64
 10  battery_capacity           980 non-null    float64
 11  fast_charging_available    980 non-null    int64  
 12  fast_charging              980 non-null    float64
 13  ram_capacity               980 non-null    float64
 14 

In [4]:
# pre-processing

#price
category = 'price'
p1 = np.percentile(df[category],1)
p2 = np.percentile(df[category],99)
df2 = df.query(f"{category} >= {p1} and {category} <= {p2}")

# #refresh_rate
# category = 'refresh_rate'
# p1 = np.percentile(df2[category],10)
# p2 = np.percentile(df2[category],90)
# df2 = df2.query(f"{category} >= {p1} and {category} <= {p2}")

#resolution
#convert to number
def evaluate_resolution(expression):
    try:
        # Replace special characters and evaluate the expression
        expression = expression['resolution'].replace(" ", "").replace("x", "*")
        return  eval(expression)
        
    except Exception as e:
        # Handle any errors that occur during evaluation
        print(f"Error evaluating expression '{expression}': {e}")
        return False
    # return expression['resolution']
    
df2['pixels'] = df2.apply(evaluate_resolution,axis=1)
#filter resolution
# category = 'pixels'
# p1 = np.percentile(df2[category],1)
# p2 = np.percentile(df2[category],100)
# df2 = df2.query(f"{category} >= {p1} and {category} <= {p2}")

# sorted(df2['price'].unique())
# df2.query("refresh_rate < 80")


C:\Users\GlitchyStar\AppData\Local\Temp\ipykernel_8776\2403701755.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pixels'] = df2.apply(evaluate_resolution,axis=1)


In [5]:
#Another method (not recommended) for finding the quartiles, doesn't take into account the volumes of mobiles having the value. only takes into account the values.

# #defining the ceiling
# max_cores = max(df2["num_cores"].unique())
# max_speed = max(df2["processor_speed"].unique())
# max_ram = max(df2["ram_capacity"].unique())
# max_price = max(df2["price"].unique())
# max_rear = max(df2["primary_camera_rear"].unique())
# max_front = max(df2["primary_camera_front"].unique())
# max_resolution = max(df2["pixels"].unique())
# max_refresh = max(df2["refresh_rate"].unique())

# max_specs = [max_cores,max_speed,max_ram,max_price,max_rear,max_front,max_front,max_resolution,max_refresh]

# #defining the ceilings
# q1_cores = np.percentile(df['num_cores'].unique(),25)
# q1_speed = np.percentile(df2["processor_speed"].unique(),25)
# q1_ram = np.percentile(df2["ram_capacity"].unique(),25)
# q1_price = np.percentile(df2["price"].unique(),25)
# q1_rear = np.percentile(df2["primary_camera_rear"].unique(),25)
# q1_front = np.percentile(df2["primary_camera_front"].unique(),25)
# q1_resolution = np.percentile(df2["pixels"].unique(),25)
# q1_refresh = np.percentile(df2["refresh_rate"],25)

# q1_specs = [q1_cores,q1_speed,q1_ram,q1_price,q1_rear,q1_front,q1_front,q1_resolution,q1_refresh]

# #defining the ceiling
# q2_cores = np.percentile(df2["num_cores"].unique(),50)
# q2_speed = np.percentile(df2["processor_speed"].unique(),50)
# q2_ram = np.percentile(df2["ram_capacity"].unique(),50)
# q2_price = np.percentile(df2["price"].unique(),50)
# q2_rear = np.percentile(df2["primary_camera_rear"].unique(),50)
# q2_front = np.percentile(df2["primary_camera_front"].unique(),50)
# q2_resolution = np.percentile(df2["pixels"].unique(),50)
# q2_refresh = np.percentile(df2["refresh_rate"],50)

# q2_specs = [q2_cores,q2_speed,q2_ram,q2_price,q2_rear,q2_front,q2_front,q2_resolution,q2_refresh]

# #defining the ceiling
# q3_cores = np.percentile(df2["num_cores"].unique(),75)
# q3_speed = np.percentile(df2["processor_speed"].unique(),75)
# q3_ram = np.percentile(df2["ram_capacity"].unique(),75)
# q3_price = np.percentile(df2["price"].unique(),75)
# q3_rear = np.percentile(df2["primary_camera_rear"].unique(),75)
# q3_front = np.percentile(df2["primary_camera_front"].unique(),75)
# q3_resolution = np.percentile(df2["pixels"].unique(),75)
# q3_refresh = np.percentile(df2["refresh_rate"],75)

# q3_specs = [q3_cores,q3_speed,q3_ram,q3_price,q3_rear,q3_front,q3_front,q3_resolution,q3_refresh]
# q1_specs,q2_specs,q3_specs,max_specs

In [6]:
#defining the ceiling
max_cores = max(df2["num_cores"].unique())
max_speed = max(df2["processor_speed"].unique())
max_ram = max(df2["ram_capacity"].unique())
max_price = max(df2["price"].unique())
max_rear = max(df2["primary_camera_rear"].unique())
max_front = max(df2["primary_camera_front"].unique())
max_resolution = max(df2["pixels"].unique())
max_refresh = max(df2["refresh_rate"].unique())

max_specs = [max_cores,max_speed,max_ram,max_price,max_rear,max_front,max_resolution,max_refresh]

#defining the ceilings
q1_cores = np.percentile(df['num_cores'],25)
q1_speed = np.percentile(df2["processor_speed"],25)
q1_ram = np.percentile(df2["ram_capacity"],25)
q1_price = np.percentile(df2["price"],25)
q1_rear = np.percentile(df2["primary_camera_rear"],25)
q1_front = np.percentile(df2["primary_camera_front"],25)
q1_resolution = np.percentile(df2["pixels"],25)
q1_refresh = np.percentile(df2["refresh_rate"],25)

q1_specs = [q1_cores,q1_speed,q1_ram,q1_price,q1_rear,q1_front,q1_resolution,q1_refresh]

#defining the ceiling
q2_cores = np.percentile(df2["num_cores"],50)
q2_speed = np.percentile(df2["processor_speed"],50)
q2_ram = np.percentile(df2["ram_capacity"],50)
q2_price = np.percentile(df2["price"],50)
q2_rear = np.percentile(df2["primary_camera_rear"],50)
q2_front = np.percentile(df2["primary_camera_front"],50)
q2_resolution = np.percentile(df2["pixels"],50)
q2_refresh = np.percentile(df2["refresh_rate"],50)

q2_specs = [q2_cores,q2_speed,q2_ram,q2_price,q2_rear,q2_front,q2_resolution,q2_refresh]

#defining the ceiling
q3_cores = np.percentile(df2["num_cores"],75)
q3_speed = np.percentile(df2["processor_speed"],75)
q3_ram = np.percentile(df2["ram_capacity"],75)
q3_price = np.percentile(df2["price"],75)
q3_rear = np.percentile(df2["primary_camera_rear"],75)
q3_front = np.percentile(df2["primary_camera_front"],75)
q3_resolution = np.percentile(df2["pixels"],75)
q3_refresh = np.percentile(df2["refresh_rate"],75)

q3_specs = [q3_cores,q3_speed,q3_ram,q3_price,q3_rear,q3_front,q3_resolution,q3_refresh]
q1_specs,q2_specs,q3_specs,max_specs

([8.0, 2.05, 4.0, 13419.25, 48.0, 8.0, 2462400.0, 60.0],
 [8.0, 2.365, 6.0, 19994.5, 50.0, 16.0, 2592000.0, 90.0],
 [8.0, 2.85, 8.0, 34999.0, 64.0, 16.0, 2600640.0, 120.0],
 [8.0, 3.22, 18.0, 163980, 200.0, 60.0, 6312960, 240])

In [119]:
 
def categorize_phone(row):
    condition_apple = 7 if row["brand_name"] == 'apple' else 0 #Apple's smooth and more fluid for same theoritical specs. So, adjusting
    condition_fast_processor = (7 + 3*(row['processor_speed']-q2_speed)/(max_speed-q2_speed)) if row['processor_speed']>=q2_speed else 7*row['processor_speed']/q2_speed
    condition_ram = (7 + 3*(row['ram_capacity']-q3_ram)/(max_ram-q3_ram)) if row['ram_capacity']>=q3_ram else 7*row['ram_capacity']/q3_ram  
    condition_fluid_display = (7 + 3*(row['refresh_rate']-q3_refresh)/(max_refresh-q3_refresh)) if row['refresh_rate']>=q3_refresh else 7*row['refresh_rate']/q3_refresh
    condition_many_cores = row["num_cores"]*10/max_cores
    condition_nfc = 5 if row["has_nfc"]==True  else 0
    condition_5g=5 if row["has_5g"]==True else 0
    condition_resolution = (7 + 4*(row['pixels']-q3_resolution)/(max_resolution-q3_resolution)) if row['pixels']>=q3_resolution else 7*row['pixels']/q3_resolution
    total = condition_5g+condition_fluid_display +  condition_fast_processor + condition_ram + condition_nfc + condition_many_cores +condition_resolution+condition_apple
    if total > 50: return "flagship"
    elif total >40: return "midrange"
    else: return "average"
    # no_of_conditions_satisfied = condition_5g.astype(int)+condition_fluid_display.astype(int) +  condition_fast_processor.astype(int) + condition_nfc.astype(int) + condition_many_cores.astype(int) +condition_resolution.astype(int)
    
def categorize_camera(row):
    if row["phone_category"] == "flagship":
        c1 = 10
    elif row["phone_category"] == "midrange":
        c1 = 8
    else:
        c1 = 5

    c2 = (9 + (row["primary_camera_rear"]-12)/(max_rear-12)) if row["primary_camera_rear"] > 12 else 9*row["primary_camera_rear"]/12
    c3 = (9 + (row["primary_camera_front"]-12)/(max_front-12)) if row["primary_camera_front"] > 10 else 9*row["primary_camera_front"]/10
    
    brand_quality = {'apple':2.5, 'google':2.5, 'samsung':1.5,'huawei':1.5}
    c4 = brand_quality[row['brand_name']] if row["brand_name"] in brand_quality else 0

    total = 10 + 10 + 10 + 2 # sum of maximum of c1 + c2 + c3 + c4
    rating = (c1+c2+c3+c4)*10.0/total
    return rating
    
def categorize_performance(row):
    c1 = row["processor_speed"]*10/max_speed
    c2 = row["num_cores"]*10/max_cores
    c3 = (7 + 3*(row['ram_capacity']-q3_ram)/(max_ram-q3_ram)) if row['ram_capacity']>=q3_ram else 7*row['ram_capacity']/q3_ram  
    c4 = 4 if row["brand_name"] == 'apple' else 0 #Apple's bionic processors usually outperform android's chips with far more memory
    
    total = 10 + 10 + 10 + 2
    rating = (c1+c2+c3+c4)*10.0/total
    return rating

In [120]:
df2['phone_category'] = df2.apply(categorize_phone, axis=1)
df2['camera_ratings'] = df2.apply(categorize_camera, axis=1)
df2['performance_ratings'] = df2.apply(categorize_performance, axis=1)


C:\Users\GlitchyStar\AppData\Local\Temp\ipykernel_8776\3225271821.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['phone_category'] = df2.apply(categorize_phone, axis=1)
C:\Users\GlitchyStar\AppData\Local\Temp\ipykernel_8776\3225271821.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['camera_ratings'] = df2.apply(categorize_camera, axis=1)
C:\Users\GlitchyStar\AppData\Local\Temp\ipykernel_8776\3225271821.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [121]:
df3=df2.sort_values(by='camera_ratings',ascending=False)
df3.head(50)

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,pixels,phone_category,camera_ratings,performance_ratings
SN,,,,,,,,,,,,,,,,,,,,,
76,google,Google Pixel 8 Pro,70990,80.0,True,True,False,google,8.0,2.85,...,1.0,android,50.0,12.0,0,0.0,4492800,flagship,9.594415,8.453416
38,apple,Apple iPhone 15 Pro Max,142990,79.0,True,True,False,bionic,6.0,3.22,...,1.0,ios,50.0,12.0,0,0.0,3566952,flagship,9.594415,8.906250
37,apple,Apple iPhone 15 Pro,130990,75.0,True,True,False,bionic,6.0,3.22,...,1.0,ios,50.0,12.0,0,0.0,2962440,flagship,9.594415,8.906250
39,apple,Apple iPhone 15 Ultra,149900,79.0,True,True,False,bionic,8.0,3.22,...,1.0,ios,48.0,12.0,0,0.0,3606840,flagship,9.591090,9.687500
28,apple,Apple iPhone 14 Pro,119990,75.0,True,True,False,bionic,6.0,3.22,...,1.0,ios,48.0,12.0,0,0.0,3013524,flagship,9.591090,8.359375
33,apple,Apple iPhone 14 Pro Max (256GB),139990,77.0,True,True,False,bionic,6.0,3.22,...,1.0,ios,48.0,12.0,0,0.0,3606840,flagship,9.591090,8.359375
31,apple,Apple iPhone 14 Pro Max,129990,76.0,True,True,False,bionic,6.0,3.22,...,1.0,ios,48.0,12.0,0,0.0,3606840,flagship,9.591090,8.359375
30,apple,Apple iPhone 14 Pro (256GB),129990,76.0,True,True,False,bionic,6.0,3.22,...,1.0,ios,48.0,12.0,0,0.0,3013524,flagship,9.591090,8.359375
70,google,Google Pixel 6 Pro (12GB RAM + 256GB),79700,89.0,True,True,False,google,8.0,2.80,...,1.0,android,50.0,11.1,0,0.0,4492800,flagship,9.588556,8.404891
